In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.signal import find_peaks
import scipy.io as sio
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]


sys.path.append("gpib_instrument_control/")
import lmx2594cmd
import gpib_instrument_control.hp_8569b
import gpib_instrument_control.hp_5340a
import gpib_instrument_control.pm_1038
import gpib_instrument_control.hp_3478a
import gpib_instrument_control.wiltron_6659a


sp = gpib_instrument_control.hp_8569b.Hp8569B()
counter=gpib_instrument_control.hp_5340a.Hp5340A()
counter.setResolution(3)
powBMeter=gpib_instrument_control.hp_3478a.Hp3478A()
pm=gpib_instrument_control.pm_1038.Pm1038(None, powBMeter, bChannelProbeCorrectionFileName='pm11-0674_correctionFactors.mat')
gen=gpib_instrument_control.wiltron_6659a.Wiltron6659A()

lmx = lmx2594cmd.Lmx2594('/dev/ttyUSB0', 320e6)
lmx.enableLockDetect(True)
lmx.setField('OUTA_PWR', 10)
lmx.setField('OUTB_PWR', 63)

Created LMX object wit fosc 320.0
Resetting LMX
Applying config
PRE DIVIDER 2
REF MULT 1
Is locked:  True
FPD is 160.000000 MHz


In [2]:
def rowStack(mtx, vec):
    if mtx is None:
        mtx=np.array(((vec)))
    else:
        mtx=np.row_stack((mtx, vec))
    return mtx

def mtxStack(mtx, vec):
    if mtx is None:
        mtx=np.array(((vec)))
    else:
        mtx=np.dstack((mtx, vec))
    return mtx

def waitForFrequencyToStablizie(approxFrequency, rate=2e6, tries=200):
        mt=0.1;
        f0=counter.readValue()
        t0=time.time();
        for i in range(tries):
            time.sleep(mt)
            f=counter.readValue()
            #print(f, np.abs(f-f0)/1e3)
            if abs(f-f0)/mt < rate and abs(f-approxFrequency) < 200e6 and f != f0:
                break
            f0=f
        else:
            print("WARNING: Frequency did not stabilize")
        return f
        #print("Frequency stabilized after", time.time()-t0, "Seconds")
    
def sweepLoRf(fif, rfs, tol=40e6, harmonic=1):
    #tol=40e6;
    peaks=[]
    pin=[]
    spectrums=None
    for rf in rfs:
        gen.setCwFreq(rf)
        time.sleep(0.5)
        rfReal=waitForFrequencyToStablizie(rf)
        #rfReal=counter.readValue()
        flo=np.abs(rfReal+fif)/harmonic
        if flo < 20e6:
            flo=20e6
        #print(rf/1e6, rfReal/1e6)
        
        lmx.setFrequency(flo)
        sp.updateTrace()
        fax, p = sp.readTrace()
        fax=np.array(fax)
        p=np.array(p)
        spectrums = rowStack(spectrums, p)
        #print(fax)
        indices = np.where(np.logical_and(fax>np.abs(fif)-tol, fax < np.abs(fif)+tol))[0]
        #print(tol, indices, fif)
        peaks.append(np.max(p[indices]))
        pin.append((pm.readChannelB(rfReal)-18))
        #print(p[indices])
        #print(fax[indices])
    return peaks, pin, spectrums, fax
        
def measureIfFrequency(fif, rfs, plot=True):
    peaks, pin = sweepLoRf(fif, rfs)
    if plot:
        plt.plot(rfs, peaks, label='output')
        plt.plot(rfs, pin, label='input')
        plt.grid(True)
        plt.legend()
    return peaks, pin, rfs       

def mixerCharacterization(ifs, rfs, mixerName, tol=40e6, harmonic=1):
    peakMtx=None
    pinMtx=None
    lossMtx=None
    spectrumCube=None
    for fif in ifs:
        peaks, inputs, spectrums, fax = sweepLoRf(fif, rfs, tol=tol, harmonic=harmonic)
        loss = np.array(inputs)-np.array(peaks)
        peakMtx=rowStack(peakMtx, peaks)
        pinMtx=rowStack(pinMtx, inputs)
        lossMtx=rowStack(lossMtx, loss)
        spectrumCube=mtxStack(spectrumCube, spectrums)
        #plt.plot(rfs, peaks, label=f'if %.2e [Hz]'%(fif))
        #plt.plot(rfs, inputs, label=f'pin %.2e [Hz]'%(fif))
        plt.plot(rfs, loss, label=f'IF %.2e [Hz]'%(fif))
    
    plt.grid(True)
    plt.xlabel('RF frequency [Hz]')
    plt.ylabel('Mixer loss [dB]')
    plt.title(f'Measurement of Mixer %s with average input power %.2f [dBm]'%(mixerName, np.mean(pinMtx)))
    plt.legend()
    saveDict={'peak':peakMtx, 'pin':pinMtx, 'loss':lossMtx, 'spectrum': spectrumCube, 'fax':fax, 'ifs':ifs, 'rfs':rfs}
    sio.savemat(mixerName+'.mat', saveDict)
    
def mapMixer(los, rfs, name):
    t0=time.time()
    rfCube=None
    rfa=[]
    rfpow=[]
    for rf in rfs:
        gen.setCwFreq(rf)
        time.sleep(3)
        arf=waitForFrequencyToStablizie(rf, rate=40e3)
        rfa.append(arf)
        rfpow.append(pm.readChannelB(arf))
        lomat=None
        for lo in los:
            lmx.setFrequency(lo)
            sp.updateTrace()
            fax, p = sp.readTrace()
            lomat=rowStack(lomat, p)
        rfCube=mtxStack(rfCube, lomat)
    saveDict={'if_axis':fax, 'rf_axis':rfa, 'lo_axis':los, 'rf_power':rfpow, 'dat':rfCube}
    sio.savemat(name+'.mat', saveDict)
    print("Total time", time.time()-t0)
    
        

In [37]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_low_band')

Total time 1548.755337715149


In [40]:
flo=np.linspace(50e6, 4e9, 50);
frf=np.linspace(50e6, 8e9, 100)
mapMixer(flo, frf, 'zem_4300_low_band_hd')

Total time 5232.677988529205


In [39]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_high_band')

Total time 2726.8662507534027


In [43]:
flo=np.linspace(50e6, 4e9, 50);
frf=np.linspace(50e6, 8e9, 100)
mapMixer(flo, frf, 'zem_4300_high_band_hd')

Total time 10025.18508720398


In [10]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_low_band_linear')

Total time 2120.732891559601


In [11]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_high_band_linear')

Total time 2668.0830183029175


In [12]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_low_band_linear_2')

Total time 2072.22514295578


In [13]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_low_band_linear_3')

Total time 2081.7983028888702


In [17]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_high_band_linear_2')

Total time 2661.5643322467804


In [18]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'zem_4300_high_band_linear_3')

Total time 2668.204257965088


In [19]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'm80lca_low_band_30dB')

Total time 2112.8507566452026


In [20]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'm80lca_high_band_30dB')

Total time 2667.163138151169


In [21]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'ML7510_low_band_30dB')

Total time 1473.7778043746948


In [22]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'ML7510_high_band_30dB')

Total time 2664.4784145355225


In [6]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'ML7510_low_band_40dB')

Total time 1468.2395315170288


In [7]:
flo=np.linspace(50e6, 4e9, 25);
frf=np.linspace(50e6, 8e9, 50)
mapMixer(flo, frf, 'ML7510_high_band_40dB')

Total time 2671.9553158283234


In [5]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'ML7510_low_band_40dB_full_freq')

Total time 5826.352819919586


In [4]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'ML7510_high_band_40dB_full_freq')

Total time 7543.985810756683


In [4]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'ML7510_low_band_30dB_full_freq')

Total time 5849.017115831375


In [8]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'ML7510_high_band_30dB_full_freq')

Total time 7490.033018827438


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'MZ9310C_low_band_30dB')

Total time 5840.506727695465


In [4]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'MZ9310C_high_band_30dB')

Total time 7504.961650848389


In [5]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'MZ9310C_reversed_low_band_30dB')

Total time 5814.74699473381


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'MZ9310C_reversed_high_band_30dB')

Total time 7509.598717689514


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'XMM302_low_band_30dB')

Total time 5871.873273134232


In [5]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'XMM302_high_band_30dB')

Total time 7531.406903028488


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 18e9, 50)
mapMixer(flo, frf, 'AT020_low_band_30dB')

Total time 3902.6397874355316


In [6]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 18e9, 50)
mapMixer(flo, frf, 'AT020_high_band_30dB')

Total time 5006.039218187332


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'AT020_low_band_30dB_full_freq')

Total time 5857.159181594849


In [7]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'AT020_high_band_30dB_full_freq')

Total time 7516.929668426514


In [4]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, '73129_low_band_30dB')

Total time 5888.988615989685


In [3]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, '73129_high_band_30dB')

Total time 7500.187081336975


In [5]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'hp10514A_low_band_30dB')

Total time 5850.0993003845215


In [7]:
flo=np.linspace(50e6, 15e9, 50);
frf=np.linspace(50e6, 24e9, 75)
mapMixer(flo, frf, 'hp10514A_high_band_30dB')

Total time 7472.746308088303
